In [ ]:
!git clone https://github.com/DenCoder618/Marks-Parser
!cd Marks-Parser

In [ ]:
import json as j
data = j.loads(open("/content/Marks-Parser/data.json", "r", encoding="utf-8").read())
print(data[0])

In [ ]:
marks = []

for period in data:
  for lesson in period["lessons"]:
    marks += lesson["marks"]
    
print(marks)

In [ ]:
ts = [m["time"] for m in marks]
marks_arr = sorted([(m["mark"][0], (m["time"]-min(ts))/(max(ts)-min(ts))) \
                    for m in marks],
                   key=lambda x: x[1])

print(marks_arr)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

colors = {"5": "tab:green",
          "4": "tab:olive",
          "3": "tab:orange",
          "2": "tab:red",
          "1": "tab:purple",
          "П": "tab:brown",
          "Б": "tab:blue",
          "Н": "tab:brown"}

for i in marks_arr:
    y, x = i
    ax.scatter(x, y, c=colors[y])

plt.show()

In [ ]:
works = []
warr = {}

for mark in marks:
  if not mark["work"] in works:
    works.append(mark["work"])
    warr.update({mark["work"]:1})
  else:
    warr.update({mark["work"]:warr[mark["work"]]+1})

works = sorted(works)
warr = {k: v for k, v in sorted(warr.items(),
                                key=lambda item: item[1],
                                reverse=True)}

print(j.dumps(warr, indent=1, ensure_ascii=False))

In [ ]:
import matplotlib.pyplot as plt

names = list(warr.keys())[2:-15]
values = list(warr.values())[2:-15]

fig, axs = plt.subplots()
axs.barh(names, values)
axs.invert_yaxis()
#fig.suptitle('Оценки по типам работ')

In [ ]:
print(j.dumps(works, indent=1, ensure_ascii=False))

Пробуем в Pandas

In [ ]:
import pandas as pd
data = {'color' : ['blue', 'green', 'yellow', 'red', 'white'],
        'object' : ['ball', 'pen', 'pencil', 'paper', 'mug'],
        'price' : [1.2, 1.0, 0.6, 0.9, 1.7]}
frame = pd.DataFrame(data)
frame

In [ ]:
print(marks)

In [ ]:
frame = pd.DataFrame(marks)
frame

In [ ]:
ts = min([m["time"] for m in marks])
arr = sorted([{"mark": m["mark"], "time": (m["time"]-ts)//100} for m in marks], 
             key=lambda x: x["time"])
arr = pd.DataFrame(arr)

In [ ]:
print(arr)

In [ ]:
pd.DataFrame(marks)

In [ ]:
ts = min([m["time"] for m in marks])
marks_arr = pd.DataFrame(sorted({"mark": [m["mark"],
                                 "time": m["time"]-ts,
                                 "date": m["date"],
                                 "work": m["work"]} \
                                 for m in marks],
                                key=lambda x: x["time"]))
print(marks_arr)

In [ ]:
import pandas as pd
import altair as alt

ts = min([m["time"] for m in marks])
arr = sorted([{"mark": m["mark"][0], "time": (m["time"]-ts)//60//60//24, "date": m["date"], "work": m["work"]} for m in marks], 
             key=lambda x: x["time"])
arr = pd.DataFrame(arr)

alt.Chart(arr).mark_circle(size=50).encode(
  x='time',
  y='work',
  color='mark',
  tooltip=["date", "work", "mark"]
).interactive() #.properties(width=800, height=300)

In [ ]:
import pandas as pd
import altair as alt

ts = min([m["time"] for m in marks])
arr = sorted([{"mark": m["mark"][0], "time": (m["time"]-ts)//60//60//24, "date": m["date"], "work": m["work"]} for m in marks], 
             key=lambda x: x["time"])
arr = pd.DataFrame(arr)

alt.Chart(arr).mark_circle(size=100).encode(
  x='time',
  y='mark',
  color='mark',
  tooltip=["date", "work", "mark"]
).properties(width=800, height=300).interactive()

In [ ]:
import altair as alt
import pandas as pd
import numpy as np

rand = np.random.RandomState(42)

df = pd.DataFrame({
    'xval': range(100),
    'yval': rand.randn(100).cumsum()
})

slider = alt.binding_range(min=0, max=100, step=1, name='cutoff:')
selector = alt.selection_single(name="SelectorName", fields=['cutoff'],
                                bind=slider, init={'cutoff': 50})

alt.Chart(df).mark_point().encode(
    x='xval',
    y='yval',
    color=alt.condition(
        alt.datum.xval < selector.cutoff,
        alt.value('red'), alt.value('blue')
    )
).add_selection(
    selector
)

In [ ]:
input_dropdown = alt.binding_select(options=['Europe','Japan','USA'])
selection = alt.selection_single(fields=['Origin'], bind=input_dropdown, name='Country of ')
color = alt.condition(selection,
                    alt.Color('Origin:N', legend=None),
                    alt.value('lightgray'))

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=color,
    tooltip='Name:N'
).add_selection(
    selection
)